In [2]:
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
import pickle
import re
from tensorflow.keras import layers , activations , models , preprocessing

In [3]:
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [4]:
model = load_model('LSTM_SEQ2SEQ_Chatbot.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 13)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 85)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 13, 200)      289600      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 85, 200)      289600      ['input_2[0][0]']                
                                                                                              

In [5]:
def make_inference_models():
    
    # Encoder model
    encoder_inputs = tf.keras.layers.Input(shape=(None,))
    encoder_embeddings = model.layers[2](encoder_inputs)
    encoder_lstm = model.layers[4]
    _, state_h_enc, state_c_enc = encoder_lstm(encoder_embeddings)
    encoder_states = [state_h_enc, state_c_enc]
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    # Decoder model
    decoder_inputs = tf.keras.layers.Input(shape=(None,))
    decoder_embeddings = model.layers[3](decoder_inputs)
    decoder_lstm = model.layers[5]
    decoder_dense = model.layers[6]
    decoder_state_input_h = tf.keras.layers.Input(shape=(200,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(200,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
        decoder_embeddings, initial_state=decoder_states_inputs)
    decoder_states = [state_h_dec, state_c_dec]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs,
                          [decoder_outputs] + decoder_states)
    
    return encoder_model, decoder_model


In [6]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=13 , padding='post')

In [6]:
# enc_model , dec_model = make_inference_models()
# user_input = input('Enter question: ')
# user_input = re.sub(r'[^\w\s]', '', user_input)  # Remove any special characters
# states_values = enc_model.predict(str_to_tokens(user_input), verbose=0)
# empty_target_seq = np.zeros( ( 1 , 1 ) )
# empty_target_seq[0, 0] = tokenizer.word_index['start']
# stop_condition = False
# decoded_translation = ''
# while not stop_condition :
#     dec_outputs , h , c =dec_model.predict([ empty_target_seq ] + states_values,verbose=0 )
#     sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
#     sampled_word = None
#     for word , index in tokenizer.word_index.items() :
#         if sampled_word_index == index :
#             decoded_translation += ' {}'.format( word )
#             sampled_word = word
    
#     if sampled_word == 'end' or len(decoded_translation.split()) > 85:
#         stop_condition = True
        
#     empty_target_seq = np.zeros( ( 1 , 1 ) )  
#     empty_target_seq[ 0 , 0 ] = sampled_word_index
#     states_values = [ h , c ] 

#     print( decoded_translation[:-3] )

In [7]:
enc_model , dec_model = make_inference_models()
for _ in range(10):
    user_input = input('Enter question: ')
    user_input = re.sub(r'[^\w\s]', '', user_input)  # Remove any special characters
    # if not all(word in tokenizer.word_index for word in user_input.split()):
    #     print("Please enter a question with words that are in the model's vocabulary.")
    states_values = enc_model.predict(str_to_tokens(user_input), verbose=0)
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c =dec_model.predict([ empty_target_seq ] + states_values,verbose=0 )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > 85:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 
    if(decoded_translation[index] < 0.5)
        print("I'm sorry, I don't know the answer to that question.")
    else:
    print( decoded_translation[:-3] )

SyntaxError: invalid syntax (1871996431.py, line 27)

In [7]:

enc_model , dec_model = make_inference_models()
while True:
    try:
        user_input = input('Enter question: ')
        user_input = re.sub(r'[^\w\s]', '', user_input)
        if len(user_input.split()) < 1:
            raise ValueError('Input is empty. Please enter a valid question.')
        elif not all(word in tokenizer.word_index for word in user_input.split()):
            raise ValueError("Please enter a question with words that are in the model's vocabulary.")
        else:
            states_values = enc_model.predict(str_to_tokens(user_input), verbose=0)
            empty_target_seq = np.zeros((1, 1))
            empty_target_seq[0, 0] = tokenizer.word_index['start']
            stop_condition = False
            decoded_translation = ''
            while not stop_condition:
                dec_outputs, h, c = dec_model.predict([empty_target_seq] + states_values, verbose=0)
                sampled_word_index = np.argmax(dec_outputs[0, -1, :])
                sampled_word = None
                for word, index in tokenizer.word_index.items():
                    if sampled_word_index == index:
                        decoded_translation += ' {}'.format(word)
                        sampled_word = word

                if sampled_word == 'end' or len(decoded_translation.split()) > 85:
                    stop_condition = True

                empty_target_seq = np.zeros((1, 1))
                empty_target_seq[0, 0] = sampled_word_index
                states_values = [h, c]

            print(decoded_translation[:-3])
    except ValueError as e:
        print('Error:', e)
        continue


Error: Please enter a question with words that are in the model's vocabulary.
 hello how can i help you today 
Error: Input is empty. Please enter a valid question.
Error: Input is empty. Please enter a valid question.
